# Sprint Challenge 4 – Previsão de Acidentes com LSTMs (Case Sompo)

**Objetivo:** Desenvolver e treinar uma Rede Neural Recorrente (LSTM) para prever padrões de acidentes nas rodovias federais, utilizando a base de dados pública da PRF. O modelo visa apoiar decisões estratégicas de prevenção e análise de riscos.

**Integrantes Big 5:**
- Lucca Phelipe Masini RM 564121
- Luiz Henrique Poss RM562177
- Luis Fernando de Oliveira Salgado RM 561401
- Igor Paixão Sarak RM 563726
- Bernardo Braga Perobeli RM 562468

---


## Passo 1: Configuração do Ambiente e Carregamento dos Dados

Para garantir a total reprodutibilidade do projeto sem a necessidade de autenticações ou uploads manuais, adotamos a seguinte estratégia para o carregamento dos dados:

1. **Hospedagem em Nuvem com Link Público:** O conjunto de dados (`dataset`) foi hospedado no Google Drive e configurado com um link de acesso público.
2. **Download Direto via URL:** O notebook utiliza o `pandas` para ler o arquivo CSV diretamente a partir de uma URL de download direto, construída a partir do link público. Isso garante que o ambiente seja independente e que o professor possa executar o código com um único clique.
3. **Importação das Bibliotecas:** Carregamos as bibliotecas Python essenciais, como `pandas`, `numpy` e `matplotlib`, para as etapas subsequentes do projeto.


In [ ]:
# --- PASSO 1: INSTALAR AS BIBLIOTECAS NECESSÁRIAS ---
!pip install openpyxl --upgrade
print("\nBibliotecas instaladas/atualizadas.")

# --- PASSO 2: IMPORTAR AS BIBLIOTECAS ---
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from io import BytesIO
import urllib.request

# --- PASSO 3: BAIXAR O ARQUIVO DIRETAMENTE DO GITHUB ---
print("Iniciando o carregamento dos dados do GitHub...")

# Link raw do arquivo no GitHub (substitua pelos seus dados)
# IMPORTANTE: Substitua SEU_USUARIO/SEU_REPO pela URL do seu repositório
github_raw_url = 'https://raw.githubusercontent.com/SEU_USUARIO/SEU_REPO/main/dados/datatran2025.xlsx'

try:
    # Substituição para um link de exemplo funcional (usando um dataset público)
    # Para usar seu próprio arquivo, atualize o URL acima
    output_filename = 'dados_acidentes.xlsx'
    
    print(f"\nBaixando dados de: {github_raw_url}")
    urllib.request.urlretrieve(github_raw_url, output_filename)
    print(f"✅ Arquivo '{output_filename}' baixado com sucesso do GitHub!")

    # --- PASSO 4: CARREGAR O DATASET USANDO pd.read_excel() ---
    df = pd.read_excel(output_filename)
    print(f"\n✅ Arquivo '{output_filename}' carregado com sucesso no pandas!")

    # Mostrando as informações para confirmar
    print("\n--- Amostra dos Dados (5 primeiras linhas) ---")
    print(df.head())
    print("\n--- Informações Gerais do DataFrame ---")
    df.info()

except Exception as e:
    print(f"\n❌ Erro ao baixar do GitHub: {e}")
    print("\n⚠️  Solução: Configure o link do GitHub corretamente:")
    print("   1. Acesse: https://github.com/seu_usuario/seu_repositorio")
    print("   2. Navegue até: dados/datatran2025.xlsx")
    print("   3. Clique em 'Raw' para obter o link direto")
    print("   4. Atualize a variável 'github_raw_url' acima com o link correto")
    print("\n   Formato do link deve ser:")
    print("   https://raw.githubusercontent.com/seu_usuario/seu_repositorio/main/dados/datatran2025.xlsx")
    raise


## Passo 2: Pré-processamento e Criação da Variável Alvo

Com os dados carregados, o próximo passo é a limpeza e a engenharia de features inicial. Esta etapa é fundamental para garantir a qualidade dos dados que alimentarão o modelo. As tarefas realizadas são:

1. **Ajuste de Tipos de Dados:** Corrigir o formato da coluna `horario`, que foi lida como texto (`object`), para um tipo de dado temporal.
2. **Criação da Variável Alvo (`target`):** Com base no objetivo do projeto, criamos uma nova coluna binária chamada `severo`. Ela receberá o valor `1` se o acidente envolveu mortos ou feridos graves, e `0` caso contrário. Esta será a variável que nosso modelo LSTM tentará prever.
3. **Seleção de Features:** Para simplificar o modelo inicial, selecionamos um subconjunto de colunas (`features`) mais relevantes para a análise.
4. **Tratamento de Dados Faltantes:** Verificamos se há valores nulos nas colunas selecionadas e aplicamos uma estratégia simples para tratá-los, garantindo que o dataset esteja completo.


In [ ]:
# Célula de Código: Limpeza e Criação do Alvo

print("Iniciando o pré-processamento...")

# 1. Ajustando a coluna 'horario' para o tipo time
# Usamos errors='coerce' para transformar horários inválidos em NaT (Not a Time)
df['horario'] = pd.to_datetime(df['horario'], format='%H:%M:%S', errors='coerce').dt.time
print("Coluna 'horario' convertida para o formato de tempo.")

# 2. Criando nossa variável alvo: Score de Gravidade Binário
# Variável binária: 1 se mortos > 0 OU feridos_graves > 0, senão 0
df['severo'] = ((df['mortos'] > 0) | (df['feridos_graves'] > 0)).astype(int)

print("Variável alvo 'severo' criada.")
print("Valor 1 para acidentes com mortos ou feridos graves.")
print("Valor 0 para demais casos.")

# 3. Selecionando as colunas que vamos usar inicialmente
# Focaremos em variáveis temporais, de localização e de contagem de pessoas/veículos
colunas_relevantes = [
    'data_inversa',
    'horario',
    'uf',
    'br',
    'km',
    'pessoas',
    'veiculos',
    'severo' # Nosso alvo!
]
df_limpo = df[colunas_relevantes].copy()
print(f"DataFrame 'df_limpo' criado com {len(colunas_relevantes)} colunas.")

# 4. Verificando e tratando valores nulos no novo DataFrame
print("\nVerificando valores nulos em 'df_limpo':")
print(df_limpo.isnull().sum())

# Como 'horario' foi a única coluna que mexemos que poderia ter nulos,
# vamos preencher os possíveis valores nulos com um horário de placeholder (meio-dia)
# Essa é uma abordagem simples, poderíamos também remover as linhas.
df_limpo['horario'].fillna(pd.to_datetime('12:00:00').time(), inplace=True)
print("\nValores nulos em 'horario' preenchidos.")

# Verificando a distribuição da nossa variável alvo
print("\n--- Distribuição da Variável Alvo 'severo' ---")
print(df_limpo['severo'].value_counts(normalize=True))

# Exibindo o resultado final do pré-processamento
print("\n--- Amostra do DataFrame Pré-processado ---")
print(df_limpo.head())
df_limpo.info()


## Passo 3: Agregação de Dados em Séries Temporais

Uma Rede Neural Recorrente (LSTM) não trabalha com registros individuais, mas sim com **sequências de dados ao longo do tempo**. Portanto, precisamos transformar nosso conjunto de dados de acidentes em uma série temporal.

A estratégia será agrupar os dados por **períodos de tempo** (semanas) e por **localização** (estado/UF). Para cada semana e cada estado, vamos calcular métricas agregadas:

- **Total de Acidentes:** A contagem total de ocorrências.
- **Total de Acidentes Severos:** A soma dos acidentes classificados como severos.
- **Proporção de Severidade:** A porcentagem de acidentes que foram severos.
- **Métricas de Volume:** Total e média de pessoas e veículos envolvidos.
- **Features Temporais:** Dia da semana, mês, identificação de fins de semana.
- **Sazonalidade:** Componentes seno e cosseno para capturar padrões anuais.

Esta abordagem nos permitirá analisar e prever como a severidade dos acidentes evolui semanalmente em cada estado, fornecendo contexto rico de informações para o modelo LSTM aprender padrões complexos e fazer previsões mais precisas.


In [ ]:
# Célula de Código: Agregação Semanal

print("Iniciando a agregação dos dados em séries temporais semanais...")

# Para facilitar a agregação baseada em data, definimos 'data_inversa' como o índice do DataFrame
df_limpo_indexed = df_limpo.set_index('data_inversa')

# Agrupando por semana (freq='W' para Weekly) e por UF.
# Para cada grupo, vamos calcular métricas agregadas:
weekly_df = df_limpo_indexed.groupby([pd.Grouper(freq='W'), 'uf']).agg(
    total_acidentes=('severo', 'count'),
    acidentes_severos=('severo', 'sum'),
    pessoas_total=('pessoas', 'sum'),
    veiculos_total=('veiculos', 'sum'),
    pessoas_media=('pessoas', 'mean'),
    veiculos_media=('veiculos', 'mean')
).reset_index()

# Criando a nossa feature principal para a série temporal: a proporção de acidentes severos
weekly_df['prop_severos'] = np.where(
    weekly_df['total_acidentes'] > 0,
    weekly_df['acidentes_severos'] / weekly_df['total_acidentes'],
    0
)

# Adicionando features temporais
weekly_df['dia_semana'] = weekly_df['data_inversa'].dt.dayofweek
weekly_df['mes'] = weekly_df['data_inversa'].dt.month
weekly_df['fim_semana'] = weekly_df['dia_semana'].isin([5, 6]).astype(int)

# Adicionando sazonalidade
weekly_df['sazonalidade_sen'] = np.sin(2 * np.pi * weekly_df['data_inversa'].dt.dayofyear / 365)
weekly_df['sazonalidade_cos'] = np.cos(2 * np.pi * weekly_df['data_inversa'].dt.dayofyear / 365)

print("\nAgregação semanal concluída com sucesso!")
print("O novo DataFrame 'weekly_df' contém o resumo semanal por estado.")

# Exibindo o resultado da transformação
print("\n--- Amostra do DataFrame Agregado Semanalmente ---")
print(weekly_df.head(10))

# Mostrando estatísticas por estado
print("\n--- Estatísticas por Estado ---")
print(weekly_df.groupby('uf').agg({
    'total_acidentes': 'sum',
    'prop_severos': 'mean'
}).sort_values('total_acidentes', ascending=False).head(10))


## Passo 4: Preparação das Sequências para a LSTM

Nesta etapa, preparamos os dados para o formato específico exigido por uma rede LSTM. O processo consiste em:

1. **Filtragem por Estados:** Utilizamos os estados com maior volume de dados para garantir uma base robusta para treinamento.
2. **Seleção de Features:** Utilizamos 6 features para enriquecer o contexto:
   - Proporção de acidentes severos (target)
   - Média de pessoas por acidente
   - Média de veículos por acidente  
   - Identificação de fim de semana
   - Componentes de sazonalidade (seno e cosseno)
3. **Normalização dos Dados:** Utilizamos o `MinMaxScaler` para normalizar todas as features para o intervalo entre 0 e 1.
4. **Criação das Janelas Temporais (Sequências):** Criamos sequências de 4 semanas para prever a próxima semana, fornecendo contexto histórico adequado para o modelo.
5. **Remodelagem (Reshape):** Ajustamos o formato para `[amostras, 4, 6]` onde temos múltiplas features por timestep.


In [ ]:
# Célula de Código: Criando as Sequências

from sklearn.preprocessing import MinMaxScaler
import numpy as np

print("Iniciando a criação das sequências para a LSTM...")

# --- 1. Filtrando os dados para os estados principais ---
# Usar os estados com mais dados para melhor treinamento
estados_principais = ['SP', 'MG', 'RJ', 'PR', 'RS', 'BA', 'CE', 'GO', 'PE', 'SC']
df_multi_estados = weekly_df[weekly_df['uf'].isin(estados_principais)].copy()
df_multi_estados = df_multi_estados.set_index('data_inversa').sort_index()

print(f"Estados selecionados: {estados_principais}")
print(f"Total de semanas: {len(df_multi_estados)}")

# --- 2. Selecionando features para o modelo ---
features_colunas = [
    'prop_severos',           # Proporção de acidentes severos (target)
    'pessoas_media',          # Média de pessoas por acidente
    'veiculos_media',         # Média de veículos por acidente
    'fim_semana',             # Se é fim de semana (0 ou 1)
    'sazonalidade_sen',       # Sazonalidade seno
    'sazonalidade_cos'        # Sazonalidade cosseno
]

# Filtrar apenas as colunas que existem
features_disponiveis = [col for col in features_colunas if col in df_multi_estados.columns]
df_features = df_multi_estados[features_disponiveis].copy()

print(f"Features selecionadas: {features_disponiveis}")

# --- 3. Normalizando os dados ---
scaler = MinMaxScaler(feature_range=(0, 1))
dados_scaled = scaler.fit_transform(df_features.values)

# --- 4. Criando as janelas temporais ---
n_passos_para_tras = 4  # 4 semanas de contexto
n_features = len(features_disponiveis)  # Múltiplas features

print(f"Janela temporal: {n_passos_para_tras} semanas")
print(f"Número de features: {n_features}")

X, y = [], []
for i in range(n_passos_para_tras, len(dados_scaled)):
    # X: sequência de n_passos_para_tras semanas com todas as features
    X.append(dados_scaled[i-n_passos_para_tras:i, :])
    # y: apenas a proporção de severos da próxima semana (primeira coluna)
    y.append(dados_scaled[i, 0])  # prop_severos é a primeira feature

# Convertendo as listas para arrays numpy
X, y = np.array(X), np.array(y)

# --- 5. Remodelando para o formato da LSTM ---
# Formato: [amostras, passos_no_tempo, n_features]
X = np.reshape(X, (X.shape[0], X.shape[1], n_features))

print("\nCriação de sequências concluída!")
print(f"Formato do array de entrada (X): {X.shape}")
print(f"Formato do array de saída (y): {y.shape}")
print(f"Número de features: {n_features}")
print(f"Janela temporal: {n_passos_para_tras} semanas")


## Passo 5: Construção e Treinamento do Modelo LSTM

Com os dados devidamente formatados em sequências, podemos finalmente construir e treinar nossa Rede Neural Recorrente (LSTM).

1. **Importação das Bibliotecas:** Importamos os componentes necessários da biblioteca `TensorFlow/Keras` para construir o modelo, incluindo callbacks como `EarlyStopping`.
2. **Definição da Arquitetura:** Modelo sequencial com camadas LSTM empilhadas:
   - **Camada LSTM:** 50 neurônios na primeira camada
   - **Camada LSTM:** 50 neurônios na segunda camada  
   - **Dropout (0.2):** Regularização para evitar overfitting
   - **Camada de Saída:** 1 neurônio com ativação linear (valores contínuos)
3. **Compilação:** Configuramos o otimizador Adam com learning rate de 0.001 e métricas adicionais (MAE).
4. **Divisão dos Dados:** Usamos 85% dos dados para treino e 15% para validação, respeitando a ordem temporal.
5. **Treinamento:** 
   - **100 épocas** com early stopping
   - **Batch size 16** padrão
   - **EarlyStopping** com paciência de 10 épocas


In [ ]:
# Célula de Código: Construindo e Treinando o Modelo

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam

print("Iniciando a construção do modelo LSTM...")

# --- 1. ARQUITETURA DO MODELO ---
model = Sequential()

# Primeira camada LSTM
model.add(LSTM(units=50, return_sequences=True, input_shape=(n_passos_para_tras, n_features)))
model.add(Dropout(0.2))

# Segunda camada LSTM
model.add(LSTM(units=50))
model.add(Dropout(0.2))

# Camada de saída
model.add(Dense(units=1, activation='linear'))

# --- 2. Compilando o modelo ---
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])

model.summary()  # Mostra um resumo da arquitetura do modelo

# --- 3. Dividindo os dados em treino e validação ---
# Usar 85% para treino e 15% para validação (mais dados para treinar)
split_index = int(len(X) * 0.85)

X_train, X_val = X[:split_index], X[split_index:]
y_train, y_val = y[:split_index], y[split_index:]

print(f"\nDados divididos em:")
print(f" - {len(X_train)} amostras de treino")
print(f" - {len(X_val)} amostras de validação")

# --- 4. CALLBACKS ---
early_stop = EarlyStopping(
    monitor='val_loss', 
    patience=10,
    verbose=1, 
    restore_best_weights=True
)

# --- 5. Treinando o modelo ---
print("\nIniciando o treinamento...")

history = model.fit(
    X_train, y_train,
    epochs=100,  # Épocas razoáveis
    batch_size=16,  # Batch size padrão
    validation_data=(X_val, y_val),
    callbacks=[early_stop],
    verbose=1
)

print("\nTreinamento concluído!")


## Passo 6: Avaliação dos Resultados e Salvamento do Modelo

A etapa final consiste em uma avaliação detalhada da performance do modelo e no salvamento do artefato para entrega.

1. **Visualização do Histórico:** Plotamos 4 gráficos para análise completa:
   - **Loss (MSE):** Curvas de treino e validação para diagnosticar overfitting
   - **MAE:** Erro médio absoluto ao longo do treinamento
   - **Previsões vs Real:** Comparação visual das previsões com dados reais
   - **Gráfico de Resíduos:** Análise da distribuição dos erros

2. **Análise de Métricas:** Calculamos múltiplas métricas para avaliação completa:
   - **MAE (Mean Absolute Error):** Erro médio absoluto
   - **MSE (Mean Squared Error):** Erro quadrático médio
   - **RMSE (Root Mean Squared Error):** Raiz do erro quadrático médio
   - **R² (Coeficiente de Determinação):** Proporção da variância explicada

3. **Salvamento do Modelo:** Salvamos o modelo no formato `.keras` para entrega.


In [ ]:
# Célula de Código: Avaliação Completa e Salvamento

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt

print("Iniciando a avaliação completa do modelo...")

# --- 1. Plotando as curvas de Loss e MAE de Treino e Validação ---
plt.figure(figsize=(15, 10))

# Subplot 1: Loss
plt.subplot(2, 2, 1)
plt.plot(history.history['loss'], label='Loss de Treino', color='blue')
plt.plot(history.history['val_loss'], label='Loss de Validação', color='red')
plt.title('Curvas de Aprendizagem - Loss (MSE)')
plt.xlabel('Épocas')
plt.ylabel('Loss (MSE)')
plt.legend()
plt.grid(True)

# Subplot 2: MAE
plt.subplot(2, 2, 2)
plt.plot(history.history['mae'], label='MAE de Treino', color='blue')
plt.plot(history.history['val_mae'], label='MAE de Validação', color='red')
plt.title('Curvas de Aprendizagem - MAE')
plt.xlabel('Épocas')
plt.ylabel('MAE')
plt.legend()
plt.grid(True)

# --- 2. Fazendo previsões no conjunto de validação ---
y_pred_scaled = model.predict(X_val, verbose=0)

# --- 3. Desnormalizando os dados para interpretação ---
# Criar array com todas as features para desnormalização
y_pred_full = np.zeros((len(y_pred_scaled), len(features_disponiveis)))
y_pred_full[:, 0] = y_pred_scaled.flatten()  # Apenas a primeira feature (prop_severos)

y_val_full = np.zeros((len(y_val), len(features_disponiveis)))
y_val_full[:, 0] = y_val  # Apenas a primeira feature (prop_severos)

# Desnormalizar
y_pred_real = scaler.inverse_transform(y_pred_full)[:, 0]
y_val_real = scaler.inverse_transform(y_val_full)[:, 0]

# --- 4. Calculando métricas ---
mae = mean_absolute_error(y_val_real, y_pred_real)
mse = mean_squared_error(y_val_real, y_pred_real)
rmse = np.sqrt(mse)
r2 = r2_score(y_val_real, y_pred_real)

print(f"\n--- Métricas de Avaliação ---")
print(f"Erro Médio Absoluto (MAE): {mae:.4f}")
print(f"Erro Quadrático Médio (MSE): {mse:.4f}")
print(f"Raiz do Erro Quadrático Médio (RMSE): {rmse:.4f}")
print(f"Coeficiente de Determinação (R²): {r2:.4f}")
print(f"Erro percentual médio: {mae*100:.2f} pontos percentuais")

# --- 5. Plotando o gráfico de Previsão vs. Real ---
plt.subplot(2, 2, 3)
plt.plot(y_val_real, label='Valores Reais', marker='o', linewidth=2, markersize=6)
plt.plot(y_pred_real, label='Previsões do Modelo', marker='x', linestyle='--', linewidth=2, markersize=6)
plt.title('Comparação: Valores Reais vs. Previsões')
plt.xlabel('Semanas (no conjunto de validação)')
plt.ylabel('Proporção de Acidentes Severos')
plt.legend()
plt.grid(True, alpha=0.3)

# --- 6. Gráfico de Resíduos ---
plt.subplot(2, 2, 4)
residuos = y_val_real - y_pred_real
plt.scatter(y_pred_real, residuos, alpha=0.7)
plt.axhline(y=0, color='red', linestyle='--')
plt.title('Gráfico de Resíduos')
plt.xlabel('Previsões')
plt.ylabel('Resíduos (Real - Predito)')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# --- 7. Análise de Performance ---
print(f"\n--- Análise de Performance ---")
print(f"Modelo treinado com {len(X_train)} amostras de treino")
print(f"Modelo validado com {len(X_val)} amostras de validação")
print(f"Features utilizadas: {len(features_disponiveis)}")
print(f"Janela temporal: {n_passos_para_tras} semanas")
print(f"Estados incluídos: {estados_principais}")

# --- 8. Salvando o modelo ---
model_filename = 'modelo_lstm_acidentes_sp.keras'
model.save(model_filename)

print(f"\nModelo salvo com sucesso no arquivo: '{model_filename}'")
print("✅ Avaliação completa finalizada!")
print("📊 Gráficos exibidos acima")
print("🎯 Modelo pronto para uso")


## Passo 7: Conclusão e Próximos Passos

### Análise dos Resultados

O treinamento do modelo LSTM para a previsão da proporção de acidentes severos demonstrou resultados promissores. O modelo foi desenvolvido com uma arquitetura robusta que inclui múltiplas camadas LSTM e técnicas de regularização adequadas.

**Características do Modelo Desenvolvido:**

- **Base de Dados Robusta:** Utilização de dados de múltiplos estados brasileiros (SP, MG, RJ, PR, RS, BA, CE, GO, PE, SC) para garantir diversidade geográfica e maior volume de dados para treinamento.

- **Features Enriquecidas:** Incorporação de 6 features que incluem componentes temporais (dia da semana, mês, fim de semana), métricas de volume (pessoas e veículos envolvidos) e componentes de sazonalidade (seno e cosseno) para capturar padrões anuais.

- **Contexto Temporal Adequado:** Janela temporal de 4 semanas fornece contexto histórico suficiente para o modelo aprender padrões temporais complexos na evolução da severidade dos acidentes.

- **Arquitetura:** Modelo com 2 camadas LSTM (50 neurônios cada) e técnicas de regularização (Dropout) para prevenir overfitting.

**Capacidades do Modelo:**

O modelo apresenta capacidade de generalização, sendo treinado com dados diversos de diferentes regiões do país. A avaliação através de múltiplas métricas (MAE, MSE, RMSE, R²) e visualizações detalhadas permite uma análise abrangente da performance do modelo.

Este modelo pode ser utilizado para apoiar decisões estratégicas de prevenção e análise de riscos nas rodovias federais, fornecendo previsões sobre a evolução da severidade dos acidentes com base em padrões históricos e características temporais.

### Próximos Passos

1. **Expandir o Dataset:** Incorporar mais estados e períodos históricos para aumentar a robustez do modelo.
2. **Features Adicionais:** Adicionar features como condições climáticas, dados de tráfego e eventos especiais.
3. **Otimização de Hiperparâmetros:** Realizar grid search para encontrar os melhores parâmetros do modelo.
4. **Deploy:** Implementar o modelo em produção para uso em tempo real.
5. **Monitoramento:** Estabelecer sistema de monitoramento contínuo da performance do modelo.
